<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%202%20-%20Exercise%20-%20Answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
import numpy as np
from keras.layers import Input, Embedding, Dot, Reshape, Add
from keras.models import Model
from keras.optimizers import Adam
import keras.backend as K
import nltk
from nltk.corpus import brown

In [7]:
nltk.download('brown')
data = brown.sents(categories=brown.categories())
len(data)

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


57340

In [0]:
sentences=[]
stopwords_ = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
punctuation = ['!','"','#','$','%','&',"'",'(',')','*','+',',','-','.','/',':',';','<','=','>','?','@','[','\\',']','^','_','`','{','|','}','~','``',"''",'--']
for sentence in data:
    for word in stopwords_:
        token=" "+word+" "
        sentence=[item.replace(token," ") for item in sentence]
    sentences.append(sentence)


In [9]:
for i in range(len(sentences)):
    sentences[i]=[item.lower() for item in sentences[i]]
    sentences[i]=[item*(len(item)>2) for item in sentences[i]]
    for pun in punctuation:
        sentences[i]=[item.replace(pun,"") for item in sentences[i]]
        sentences[i]=[item for item in sentences[i] if item]
print(sentences[100])

['daniel', 'personally', 'led', 'the', 'fight', 'for', 'the', 'measure', 'which', 'had', 'watered', 'down', 'considerably', 'since', 'its', 'rejection', 'two', 'previous', 'legislatures', 'public', 'hearing', 'before', 'the', 'house', 'committee', 'revenue', 'and', 'taxation']


In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
vocab_size=10000
vector_dim=200
maxlen=20
windowSize=3

In [0]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

In [0]:
sequences = tokenizer.texts_to_sequences(sentences)
sequences_padded = pad_sequences(sequences, padding='post', maxlen=maxlen)


In [24]:
print(sentences[4000])
print(sequences[4000])
print(sequences_padded[4000])
print(len(sentences[4000]))
print(len(sequences[4000]))
print(len(sequences_padded[4000]))

['ruths', 'day', 'and', 'until', 'this', 'year', 'the', 'schedule', 'was', '154', 'games']
[9177, 108, 3, 161, 9, 117, 2, 2968, 5, 1, 1826]
[9177  108    3  161    9  117    2 2968    5    1 1826    0    0    0
    0    0    0    0    0    0]
11
11
20


In [0]:
from collections import defaultdict
co_matrix = defaultdict(lambda: defaultdict(int))

In [0]:
for sentence in sequences_padded:
    sentence_size=len(sentence)
    for i in range(sentence_size):
        for distance in range(1,windowSize+1):
            if i+distance<sentence_size:
                if sentence[i]>sentence[i+distance]:
                    first=sentence[i+distance]
                    second=sentence[i]
                else:
                    second=sentence[i+distance]
                    first=sentence[i]
                co_matrix[first][second]+=1.0/distance

In [0]:
first,second,freqs=[],[],[]
for first_id in co_matrix.keys():
    for second_id in co_matrix[first_id].keys():
        freq=co_matrix[first_id][second_id]
        first.append(first_id)
        second.append(second_id)
        freqs.append(freq)
        first.append(second_id)
        second.append(first_id)
        freqs.append(freq)
first=np.array(first)
second=np.array(second)
freqs=np.array(freqs)

In [28]:
print(np.shape(first))
print(np.shape(second))
print(np.shape(freqs))

(1306226,)
(1306226,)
(1306226,)


In [0]:
#time for modelling yay!
#shamelessly copied loss haha
X_MAX = 100
a = 3.0 / 4.0
def customLoss(y_true,y_pred):
    return K.sum(K.pow(K.clip(y_true / X_MAX, 0.0, 1.0), a) * K.square(y_pred - K.log(y_true)), axis=-1)

In [0]:
input_target = Input((1,))
input_context = Input((1,))

central_embedding = Embedding(vocab_size, vector_dim, input_length=1,name='cen_e')
central_bias = Embedding(vocab_size, 1, input_length=1)

context_embedding = Embedding(vocab_size, vector_dim, input_length=1,name='con_e')
context_bias = Embedding(vocab_size, 1, input_length=1)

vector_target = central_embedding(input_target)
vector_context = context_embedding(input_context)

bias_target = central_bias(input_target)
bias_context = context_bias(input_context)

dot_product = Dot(axes=-1)([vector_target, vector_context])
dot_product = Reshape((1, ))(dot_product)
bias_target = Reshape((1,))(bias_target)
bias_context = Reshape((1,))(bias_context)

prediction = Add()([dot_product, bias_target, bias_context])

model = Model(inputs=[input_target, input_context], outputs=prediction)
model.compile(loss=customLoss, optimizer=Adam())

In [0]:
epochs=50
batch_size=256

In [32]:
model.fit([first, second], freqs, epochs=epochs, batch_size=batch_size)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/50
1306226/1306226 [==============================] - 19s 15us/step - loss: 0.0505
Epoch 2/50
1306226/1306226 [==============================] - 18s 14us/step - loss: 0.0186
Epoch 3/50
1306226/1306226 [==============================] - 18s 14us/step - loss: 0.0123
Epoch 4/50
1306226/1306226 [==============================] - 17s 13us/step - loss: 0.0079
Epoch 5/50
1306226/1306226 [==============================] - 17s 13us/step - loss: 0.0055
Epoch 6/50
1306226/1306226 [==============================] - 17s 13us/step - loss: 0.0042
Epoch 7/50
1306226/1306226 [==============================] - 17s 13us/step - loss: 0.0035
Epoch 8/50
1306226/1306226 [==============================] - 17s 13us/step - loss: 0.0030
Epoch 9/50
1306226/1306226 [==============================] - 17s 13us/step - loss: 0.0027
Epoch 10/50
1306226/1306226 [==============================] - 17s 13us/step - loss: 0.0024
Epoch 11/50
1306226/1306226 [==============================] - 17s 13us/step - loss: 0.00

In [33]:
cen_e=central_embedding.get_weights()
con_e=context_embedding.get_weights()
print(np.shape(cen_e))
print(np.shape(con_e))

(1, 10000, 200)
(1, 10000, 200)


In [0]:
 emb_matrix = np.zeros((vocab_size,vector_dim))
 word_2_vec={}

In [0]:
for i in range(vocab_size):
    emb_matrix[i]=(con_e[0][i]+cen_e[0][i])/2
    word_2_vec[str(tokenizer.index_word[1+i])]=emb_matrix[i]

In [36]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv \
    -O /tmp/bbc-text.csv
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_portion = .8

--2020-05-23 08:27:01--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c09::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [application/octet-stream]
Saving to: ‘/tmp/bbc-text.csv’

/tmp/bbc-text.csv   100%[===================>]   4.82M  --.-KB/s    in 0.04s   

2020-05-23 08:27:01 (122 MB/s) - ‘/tmp/bbc-text.csv’ saved [5057493/5057493]



In [37]:
sentences = []
labels = []
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
print(len(stopwords))
# Expected Output
# 153

153


In [38]:
with open("/tmp/bbc-text.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[0])
        sentence = row[1]
        for word in stopwords:
            token = " " + word + " "
            sentence = sentence.replace(token, " ")
        sentences.append(sentence)

print(len(labels))
print(len(sentences))
print(sentences[0])

2225
2225
tv future hands viewers home theatre systems  plasma high-definition tvs  digital video recorders moving living room  way people watch tv will radically different five years  time.  according expert panel gathered annual consumer electronics show las vegas discuss new technologies will impact one favourite pastimes. us leading trend  programmes content will delivered viewers via home networks  cable  satellite  telecoms companies  broadband service providers front rooms portable devices.  one talked-about technologies ces digital personal video recorders (dvr pvr). set-top boxes  like us s tivo uk s sky+ system  allow people record  store  play  pause forward wind tv programmes want.  essentially  technology allows much personalised tv. also built-in high-definition tv sets  big business japan us  slower take off europe lack high-definition programming. not can people forward wind adverts  can also forget abiding network channel schedules  putting together a-la-carte entertai

In [39]:
train_size = int(len(sentences) * training_portion)

train_sentences = sentences[:train_size]
train_labels = labels[:train_size]

validation_sentences = sentences[train_size:]
validation_labels = labels[train_size:]

print(train_size)
print(len(train_sentences))
print(len(train_labels))
print(len(validation_sentences))
print(len(validation_labels))

1780
1780
1780
445
445


In [40]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

print(len(train_sequences[0]))
print(len(train_padded[0]))

print(len(train_sequences[1]))
print(len(train_padded[1]))

print(len(train_sequences[10]))
print(len(train_padded[10]))

449
120
200
120
192
120


In [41]:
validation_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(validation_sequences, padding=padding_type, maxlen=max_length)

print(len(validation_sequences))
print(validation_padded.shape)

445
(445, 120)


In [42]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))

print(training_label_seq[0])
print(training_label_seq[1])
print(training_label_seq[2])
print(training_label_seq.shape)

print(validation_label_seq[0])
print(validation_label_seq[1])
print(validation_label_seq[2])
print(validation_label_seq.shape)

[4]
[2]
[1]
(1780, 1)
[5]
[4]
[3]
(445, 1)


In [47]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import Flatten
from keras.layers import Embedding,LSTM,Bidirectional
import keras
model_i=Sequential()
model_i.add(Embedding(vocab_size, vector_dim, weights=[emb_matrix], input_length=120, trainable=False))
model_i.add(Bidirectional(LSTM(128)))
model_i.add(Dropout(0.2))
model_i.add(keras.layers.Dense(64, activation='relu'))
model_i.add(Dropout(0.3))
model_i.add(keras.layers.Dense(6, activation='softmax'))
model_i.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_i.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 120, 200)          2000000   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               336896    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 390       
Total params: 2,353,734
Trainable params: 353,734
Non-trainable params: 2,000,000
______________________________________

In [48]:
num_epochs = 60
history = model_i.fit(train_padded, training_label_seq, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq), verbose=2)

Train on 1780 samples, validate on 445 samples
Epoch 1/60
 - 20s - loss: 1.6414 - accuracy: 0.2500 - val_loss: 1.5568 - val_accuracy: 0.2809
Epoch 2/60
 - 19s - loss: 1.4169 - accuracy: 0.3994 - val_loss: 1.3112 - val_accuracy: 0.4067
Epoch 3/60
 - 20s - loss: 1.2332 - accuracy: 0.5079 - val_loss: 1.1254 - val_accuracy: 0.5775
Epoch 4/60
 - 20s - loss: 1.0139 - accuracy: 0.6185 - val_loss: 1.1967 - val_accuracy: 0.5169
Epoch 5/60
 - 19s - loss: 0.8215 - accuracy: 0.6955 - val_loss: 0.9697 - val_accuracy: 0.6494
Epoch 6/60
 - 19s - loss: 0.6947 - accuracy: 0.7545 - val_loss: 0.9377 - val_accuracy: 0.6719
Epoch 7/60
 - 20s - loss: 0.5934 - accuracy: 0.7860 - val_loss: 0.8674 - val_accuracy: 0.6652
Epoch 8/60
 - 19s - loss: 0.4646 - accuracy: 0.8517 - val_loss: 0.7247 - val_accuracy: 0.7506
Epoch 9/60
 - 19s - loss: 0.4073 - accuracy: 0.8618 - val_loss: 0.7833 - val_accuracy: 0.7596
Epoch 10/60
 - 19s - loss: 0.3136 - accuracy: 0.9034 - val_loss: 0.6768 - val_accuracy: 0.7685
Epoch 11/60


In [49]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_sentence(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])
e = model_i.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 200)


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')